In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!pip install transformers==3.5.0
!pip install torch==1.4.0

     |████████████████████████████████| 753.4MB 17kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101


In [ ]:
import os
import time
import datetime
import torch
from torch.utils.data import Dataset



import argparse
import logging
import os
import random

import numpy as np

import torch
from torch.utils.data import DataLoader

from transformers import (
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

model_name = 'google/pegasus-cnn_dailymail'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
from transformers.modeling_bart import shift_tokens_right

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import tensorflow as tf
train_dataset_fp = tf.keras.utils.get_file('/content/gdrive/MyDrive/H2_B2I_14/Preprocessed_Dataset_Summarization.csv','Preprocessed_Dataset_Summarization.csv')
batch_size = 2

train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    select_columns = ['text','ctext'],
    num_epochs=1)

In [ ]:
for i,batch in enumerate(train_dataset):
  break

In [ ]:
def label_smoothed_nll_loss(lprobs, target, epsilon, ignore_index=-100):
    """From fairseq"""
    if target.dim() == lprobs.dim() - 1:
        target = target.unsqueeze(-1)
    nll_loss = -lprobs.gather(dim=-1, index=target)
    smooth_loss = -lprobs.sum(dim=-1, keepdim=True)
    if ignore_index is not None:
        pad_mask = target.eq(ignore_index)
        nll_loss.masked_fill_(pad_mask, 0.0)
        smooth_loss.masked_fill_(pad_mask, 0.0)
    else:
        nll_loss = nll_loss.squeeze(-1)
        smooth_loss = smooth_loss.squeeze(-1)

    nll_loss = nll_loss.sum()  # mean()? Scared to break other math.
    smooth_loss = smooth_loss.sum()
    eps_i = epsilon / lprobs.size(-1)
    loss = (1.0 - epsilon) * nll_loss + eps_i * smooth_loss
    return loss, nll_loss


In [ ]:
num_train_epochs = 5
#model.resize_token_embeddings(len(tokenizer))
weight_decay =0.0
learning_rate = 1e-4 
adam_epsilon = 1e-8
warmup_steps = 0
t_total = (607276 // 2 ) 

In [ ]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
    )
if(os.path.exists('/content/gdrive/MyDrive/H2_B2I_14/Pegasus')):
  optimizer.load_state_dict(torch.load(os.path.join('/content/gdrive/MyDrive/H2_B2I_14/Pegasus', 'optimizer.pt')))
  scheduler.load_state_dict(torch.load(os.path.join('/content/gdrive/MyDrive/H2_B2I_14/Pegasus', 'scheduler.pt')))

In [ ]:
output_dir='/content/gdrive/MyDrive/H2_B2I_14/Pegasus'

In [ ]:
pad_token_id = tokenizer.pad_token_id
epochs = 5
for epoc in range(epochs):
  t0 = time.time()
  print("")
  print('======== Epoch {} ========'.format(epoc+1))
  model.train()
  total_train_loss = 0
  for i,batch in enumerate(train_dataset):
    title = []
    body = []
    for item in batch['text'].numpy():
      title.append(item.decode('utf-8'))
    for item in batch['ctext'].numpy():
      body.append(item.decode('utf-8')) 

    batch_tokens = tokenizer.prepare_seq2seq_batch(body,title,max_length=320,max_target_length=60,truncation=True,padding='max_length').to(torch_device)
    decoder_input_ids = shift_tokens_right(batch_tokens['labels'], pad_token_id)
    outputs = model(batch_tokens['input_ids'], attention_mask=batch_tokens['attention_mask'], decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    lprobs = torch.nn.functional.log_softmax(lm_logits, dim=-1)
    loss, nll_loss = label_smoothed_nll_loss(
                lprobs, batch_tokens['labels'],0.1, ignore_index=pad_token_id
            )
    total_train_loss += loss.item()
  
    optimizer.zero_grad()
    loss.backward()
    #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    if i%500 == 0:
      print("Batch :" + str(i)+ "\tTraining Loss(Un-normalized_Additive): " +str(total_train_loss))
    if (i+1) % 10000 == 0:
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
        torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt')) 
  training_time = format_time(time.time() - t0)
  avg_train_loss = total_train_loss / 1050994
  print("Training loss for epoch "+str(epoc+1)+": "+str(avg_train_loss))
  print("time : {}".format(training_time))
  
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(output_dir)
  tokenizer.save_pretrained(output_dir)
  torch.save(optimizer.state_dict(), os.path.join(output_dir, 'optimizer.pt'))
  torch.save(scheduler.state_dict(), os.path.join(output_dir, 'scheduler.pt'))


======== Epoch 1 ========
batch :0Training Loss: 110.03550720214844
batch :500Training Loss: 58093.88905334473
batch :1000Training Loss: 116078.9450263977
batch :1500Training Loss: 174389.08918762207
traing loss epoch1:0.19262331801715735
time : 0:17:07

======== Epoch 2 ========
batch :0Training Loss: 109.35237884521484
batch :500Training Loss: 57003.09384918213
batch :1000Training Loss: 115199.2033996582
batch :1500Training Loss: 171935.46877288818
traing loss epoch2:0.1892860587697253
time : 0:17:14

======== Epoch 3 ========
batch :0Training Loss: 128.6055908203125
batch :500Training Loss: 56279.194396972656
batch :1000Training Loss: 111771.41904449463
batch :1500Training Loss: 168799.98818588257
traing loss epoch3:0.18576116651559138
time : 0:17:10

======== Epoch 4 ========
batch :0Training Loss: 107.75408935546875
batch :500Training Loss: 55529.86949920654
batch :1000Training Loss: 111503.86494445801
batch :1500Training Loss: 166717.5108795166
traing loss epoch4:0.1830949294598